In [1]:
import os
import sys
import json
from os.path import join, basename, dirname

import numpy as np
import pandas as pd

from metric_learning_evaluator.index.agent import IndexAgent
from metric_learning_evaluator.data_tools.embedding_container import EmbeddingContainer
from metric_learning_evaluator.utils.template_parser import InferenceResult


In [2]:
data_dir = '/home/kv_zhao/nist-e2e/metric-learning-evaluator/feature-examples/featobj_mergeV1_D40kv2_RMG/'

In [ ]:
RMG = InferenceResult('/home/kv_zhao/nist-e2e/outcomes/MERGE_V1/D40kv2_RMG_iv1_pv1/')

In [3]:
container = EmbeddingContainer()
container.load(data_dir)

Container:embedding_container created
Load embedding container from feat_obj format
/home/kv_zhao/nist-e2e/metric-learning-evaluator/feature-examples/featobj_mergeV1_D40kv2_RMG//embeddings.npy is loaded
/home/kv_zhao/nist-e2e/metric-learning-evaluator/feature-examples/featobj_mergeV1_D40kv2_RMG//label_ids.npy is loaded
/home/kv_zhao/nist-e2e/metric-learning-evaluator/feature-examples/featobj_mergeV1_D40kv2_RMG//filename_strings.npy is loaded
/home/kv_zhao/nist-e2e/metric-learning-evaluator/feature-examples/featobj_mergeV1_D40kv2_RMG//probabilities.npy is loaded
/home/kv_zhao/nist-e2e/metric-learning-evaluator/feature-examples/featobj_mergeV1_D40kv2_RMG//label_names.npy is loaded
/home/kv_zhao/nist-e2e/metric-learning-evaluator/feature-examples/featobj_mergeV1_D40kv2_RMG//landmarks.npy is loaded
/home/kv_zhao/nist-e2e/metric-learning-evaluator/feature-examples/featobj_mergeV1_D40kv2_RMG//instance_ids.npy is loaded
container size: 10000 -> 134281
embedding size: 0 -> 1024
probability siz

In [17]:
instance_ids = container.instance_ids
all_embeddings = container.get_embedding_by_instance_ids(instance_ids)
agent = IndexAgent('HNSW', instance_ids, all_embeddings, distance_measure='ip')

HNSW Index Agent is initialized with 134281 features


In [4]:
label_ids = container.label_ids
sample_label_id = label_ids[3]

In [30]:
sample_inst_ids = container.get_instance_ids_by_label_ids(sample_label_id)
sample_embeddings = container.get_embedding_by_instance_ids(sample_inst_ids)

In [31]:
print(len(sample_inst_ids))

5


In [32]:
idxs, sims = agent.search(sample_embeddings, top_k=10, is_similarity=True)

In [33]:
sims

array([[2.       , 1.4887882, 1.4800723, 1.47902  , 1.4719896, 1.3325983,
        1.298273 , 1.295822 , 1.2915524, 1.2859523],
       [2.       , 1.8132361, 1.6747476, 1.6252718, 1.4719896, 1.3601923,
        1.3281094, 1.2833375, 1.2816005, 1.2815654],
       [1.9999999, 1.8132361, 1.7025867, 1.661875 , 1.4887882, 1.3477598,
        1.3217814, 1.3063887, 1.2918242, 1.2916085],
       [2.       , 1.7025867, 1.634887 , 1.6252718, 1.4800723, 1.3356001,
        1.3096268, 1.3019067, 1.2917447, 1.2852807],
       [2.       , 1.6747476, 1.661875 , 1.634887 , 1.47902  , 1.3090386,
        1.2818949, 1.2800608, 1.2779536, 1.2695997]], dtype=float32)

In [34]:
print(idxs[0])
print(sample_inst_ids)

[    3     5     6     7     4 96728 96731 20703 21788 20710]
[3, 4, 5, 6, 7]
